In [ ]:
import findspark
import warnings
warnings.filterwarnings('ignore')

In [ ]:
findspark.init('../../../../../../../Program Files/Spark/spark-3.2.0-bin-hadoop3.2')

In [ ]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc
from collections import namedtuple
from IPython import display

import time
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
sc = SparkContext()

In [ ]:
ssc = StreamingContext(sc, 10)
sqlContext = SQLContext(sc)

In [ ]:
socket_stream = ssc.socketTextStream('127.0.0.1', 9998)

In [ ]:
lines = socket_stream.window(20)

In [ ]:
fields = ("tag", "count")
Tweet = namedtuple('Tweet', fields)

In [ ]:
(lines.flatMap(lambda text: text.split(" ")) # Splits to a list
      .filter(lambda word: word.lower().startwith("#")) # Checks for hashtag calls
      .map(lambda word: (word.lower(), 1)) # Lower cases the word
      .reduceByKey(lambda a, b: a + b) # Reduces
      .map(lambda rec: Tweet(rec[0], rec[1])) # Stores in a Tweet Object
      .foreachRDD(lambda rdd: rdd.toDF().sort(desc("count")) # Sorts Them in a DF
      .limit(10).creteOrReplaceTempView("Tweets"))) # Registers to a table.

In [ ]:
ssc.start()

In [ ]:
count = 0
while count < 10:
    time.sleep(3)
    top_10_tweets = sqlContext.sql('Select tag, count from tweets')
    df_top10 = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    plt.figure(figsize=(10,8))
    sns.barplot(data=df_top10, x='count', y='tag')
    plt.show()
    count = count + 1

In [ ]:
ssc.stop()